# Data cleansing and EDA (refactored v3)

This notebook is a cleaned and reorganized version of the original. It contains consolidated helpers and a clear workflow:
1. Imports & configuration
2. Load raw data
3. Cleaning helpers and application
4. Country & institution standardization (UNSD / ROR)
5. OECD -> major scientific discipline mapping
6. Export cleaned data

Keep this file as the readable, single-pass version to run end-to-end.

## 1) Imports and configuration

In [1]:
import os, re
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from typing import Optional, Tuple, Any

# Paths (update if needed)
RAW_XLSX = r"raw_data/20251024_raw data BioStruct_map client project_final.xlsx"
UNSD_PATH = r"processed_data/unsd_methodology_clean.csv"
ROR_PATH  = r"processed_data/ror_data_clean.csv"
OECD_MAPPING_XLSX = r"raw_data/OECD schema to Web of Science Category Mapping 2022.xlsx"
OUT_PATH = r"processed_data/cleansed_biostruct_data_v3.csv"
os.makedirs("processed_data", exist_ok=True)

## 2) Load raw data

In [2]:
# Read the Excel file; change path above if needed
df = pd.read_excel(RAW_XLSX)
print('Loaded', len(df), 'rows and', len(df.columns), 'columns')
df.head()

Loaded 308 rows and 9 columns


ID  Workshop Evaluation Gender       Status  \
0  9208  Cameroon          R   Male      Pre-Doc   
1  4358     Kenya          R   Male      Pre-Doc   
2  7084     Kenya          S   Male  PhD Student   
3  8849      Mali          R   Male      Pre-Doc   
4  3158     Kenya          R   Male      Pre-Doc   

                               Scientific_Discipline  Institutional_ID  \
0                                  Organic Chemistry               NaN   
1                                           Pharmacy               NaN   
2                                Medicinal Chemistry               NaN   
3                                                 na               NaN   
4  Bioinformatics, Zoology,\nCell Biology and Gen...               NaN   

                  Primary_ institutional_affiliation  \
0         Prince Abubakar\n Audu University, Nigeria   
1                                                  -   
2               Nazi Boni University,\n Burkina Faso   
3  Université des Sciences, des Techniques et des...   
4                   Lagos State University,\nNigeria   

  Country_of_primary_affiliation  
0                        Nigeria  
1                             na  
2                   Burkina Faso  
3                           Mali  
4                        Nigeria

## 3) Cleaning helpers and apply

In [3]:
def s_trim(x: Any) -> Optional[str]:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    t = str(x).strip()
    return t if t != '' else None

def collapse_ws(s: str) -> str:
    return re.sub(r'\s+', ' ', str(s)).strip()

def ratio(a: str, b: str) -> float:
    return SequenceMatcher(None, str(a).strip(), str(b).strip()).ratio()

def clean_string_columns(df: pd.DataFrame, replace_with='Unknown') -> pd.DataFrame:
    """Normalize whitespace and map common non-values to a single token (default 'Unknown')."""
    unsuitable_patterns = set(['', '-', 'na', 'n/a', 'null', 'none', 'unknown', 'nan'])
    for col in df.columns:
        if df[col].dtype == object or df[col].dtype.name == 'string':
            ser = df[col].astype(str).fillna('').map(lambda x: collapse_ws(x))
            ser_low = ser.str.lower().str.strip()
            mask = ser_low.isin(unsuitable_patterns) | ser_low.str.fullmatch(r'[-–—]+') | (ser_low.str.len() == 0)
            ser = ser.mask(mask, replace_with)
            df[col] = ser
    return df

# Apply cleaning
df = clean_string_columns(df, replace_with='Unknown')
print('Applied string cleaning; sample:')
df.head()

Applied string cleaning; sample:


ID  Workshop Evaluation Gender       Status  \
0  9208  Cameroon          R   Male      Pre-Doc   
1  4358     Kenya          R   Male      Pre-Doc   
2  7084     Kenya          S   Male  PhD Student   
3  8849      Mali          R   Male      Pre-Doc   
4  3158     Kenya          R   Male      Pre-Doc   

                               Scientific_Discipline  Institutional_ID  \
0                                  Organic Chemistry               NaN   
1                                           Pharmacy               NaN   
2                                Medicinal Chemistry               NaN   
3                                            Unknown               NaN   
4  Bioinformatics, Zoology, Cell Biology and Gene...               NaN   

                  Primary_ institutional_affiliation  \
0           Prince Abubakar Audu University, Nigeria   
1                                            Unknown   
2                 Nazi Boni University, Burkina Faso   
3  Université des Sciences, des Techniques et des...   
4                    Lagos State University, Nigeria   

  Country_of_primary_affiliation  
0                        Nigeria  
1                        Unknown  
2                   Burkina Faso  
3                           Mali  
4                        Nigeria

## 4) Country & institution standardization (UNSD / ROR)

In [4]:
# Load reference tables if available
unsd = pd.read_csv(UNSD_PATH, encoding='utf-8-sig') if os.path.exists(UNSD_PATH) else pd.DataFrame()
ror  = pd.read_csv(ROR_PATH,  encoding='utf-8-sig') if os.path.exists(ROR_PATH) else pd.DataFrame()

# Normalize reference tables
def norm_ref_table(tbl: pd.DataFrame) -> pd.DataFrame:
    if tbl.empty: return tbl
    for col in tbl.columns:
        if tbl[col].dtype == object:
            tbl[col] = tbl[col].astype(str).map(lambda x: collapse_ws(x) if x and x.lower() != 'nan' else None)
    return tbl

unsd = norm_ref_table(unsd)
ror = norm_ref_table(ror)

# Build lookup lists
unsd_by_iso = {}
unsd_countries = []
ror_countries = []
if not unsd.empty:
    unsd_by_iso = {str(r['iso_alpha2']).strip().upper(): (str(r['country_or_area']).strip(), str(r.get('subregion')).strip() if pd.notna(r.get('subregion')) else None) for _, r in unsd.iterrows() if pd.notna(r.get('iso_alpha2'))}
    unsd_countries = [str(x).strip() for x in unsd['country_or_area'].dropna().tolist()]
if not ror.empty and 'country_name' in ror.columns:
    ror_countries = [str(x).strip() for x in ror['country_name'].dropna().tolist()]

unsd_countries_set = set(unsd_countries)
ror_countries_set = set(ror_countries)

def fuzzy_lookup_country(name: Optional[str], code: Optional[str]) -> Tuple[Optional[str], Optional[str], Optional[str]]:
    name = s_trim(name); code = s_trim(code)
    if code:
        up = code.upper()
        if up in unsd_by_iso:
            nm, reg = unsd_by_iso[up]
            return (s_trim(nm), up, s_trim(reg))
    if name and name in unsd_countries_set:
        rows = unsd[unsd['country_or_area'] == name]
        if not rows.empty:
            iso = s_trim(str(rows.iloc[0].get('iso_alpha2'))) if 'iso_alpha2' in rows.columns else None
            reg = s_trim(str(rows.iloc[0].get('subregion'))) if 'subregion' in rows.columns else None
            return (name, iso, reg)
    # fuzzy match in UNSD list
    if name:
        best, best_nm = 0.0, None
        for cand in unsd_countries:
            r = ratio(name, cand)
            if r > best: best, best_nm = r, cand
        if best_nm and best >= 0.9:
            rows = unsd[unsd['country_or_area'] == best_nm]
            iso = s_trim(str(rows.iloc[0].get('iso_alpha2'))) if 'iso_alpha2' in rows.columns else None
            reg = s_trim(str(rows.iloc[0].get('subregion'))) if 'subregion' in rows.columns else None
            return (best_nm, iso, reg)
    return (name, code.upper() if code else None, None)

# Add working columns to main df
df['country_raw']     = df.get('Country_of_primary_affiliation', pd.Series([None]*len(df))).map(lambda x: collapse_ws(x) if x else None)
df['institution_raw'] = df.get('Primary_ institutional_affiliation', pd.Series([None]*len(df))).map(lambda x: collapse_ws(x) if x else None)

# Standardize country + region
country_name_std_list, country_code_std_list, region_list = [], [], []
for _, row in df.iterrows():
    cn_std, cc_std, reg = fuzzy_lookup_country(row.get('country_raw'), None)
    country_name_std_list.append(cn_std)
    country_code_std_list.append(cc_std)
    region_list.append(reg)

df['country_name_std'] = country_name_std_list
df['country_code_std'] = country_code_std_list
df['region'] = region_list

# Institution standardization (best-effort using ROR)
if not ror.empty:
    ror_work = ror.copy()
    ror_work['aliases_list']  = ror_work['aliases'].apply(lambda v: [seg.strip() for seg in str(v).split(';')] if 'aliases' in ror_work.columns else []) if 'aliases' in ror_work.columns else [[] for _ in range(len(ror_work))]
    ror_work['acronyms_list'] = ror_work['acronyms'].apply(lambda v: [seg.strip() for seg in str(v).split(';')] if 'acronyms' in ror_work.columns else []) if 'acronyms' in ror_work.columns else [[] for _ in range(len(ror_work))]

    def ror_filter_by_country(code: Optional[str], name: Optional[str]) -> pd.DataFrame:
        code = s_trim(code); name = s_trim(name)
        if code:
            dfc = ror_work[ror_work['country_code'].astype(str).str.upper().str.strip() == code.upper()]
            if not dfc.empty: return dfc
        if name:
            dfc = ror_work[ror_work['country_name'].astype(str).str.strip() == name]
            return dfc
        return ror_work.iloc[0:0]

    def choose_standard_name(inst_name: Optional[str], country_name_std: Optional[str], country_code_std: Optional[str], fuzzy_threshold: float = 0.92) -> str:
        raw = s_trim(inst_name)
        if not raw: return ''
        raw_eq = collapse_ws(raw)
        cand = ror_filter_by_country(country_code_std, country_name_std)
        if cand.empty: return raw
        exact = cand[cand['institution_name'].apply(lambda x: collapse_ws(str(x)) == raw_eq)]
        if not exact.empty: return str(exact.iloc[0]['institution_name']).strip()
        def alias_exact(row):
            for a in row.get('aliases_list', []):
                if collapse_ws(a) == raw_eq: return True
            return False
        alias_rows = cand[cand.apply(alias_exact, axis=1)]
        if not alias_rows.empty: return str(alias_rows.iloc[0]['institution_name']).strip()
        def acro_exact(row):
            for a in row.get('acronyms_list', []):
                if collapse_ws(a) == raw_eq: return True
            return False
        acro_rows = cand[cand.apply(acro_exact, axis=1)]
        if not acro_rows.empty: return str(acro_rows.iloc[0]['institution_name']).strip()
        best_score, best_name = 0.0, None
        for _, r in cand.iterrows():
            nm = s_trim(r.get('institution_name'))
            if nm:
                sc = ratio(raw, nm)
                if sc > best_score: best_score, best_name = sc, nm
            for a in r.get('aliases_list', []):
                if not a: continue
                sc = ratio(raw, a)
                if sc > best_score: best_score, best_name = sc, r.get('institution_name')
            for a in r.get('acronyms_list', []):
                if not a: continue
                sc = ratio(raw, a)
                if sc > best_score: best_score, best_name = sc, r.get('institution_name')
        if best_name is not None and best_score >= fuzzy_threshold: return str(best_name).strip()
        return raw

    mapped_names = []
    for _, row in df.iterrows():
        mapped = choose_standard_name(row.get('institution_raw'), row.get('country_name_std'), row.get('country_code_std'))
        mapped_names.append(mapped)
    df['institution_name_standardized'] = mapped_names
else:
    df['institution_name_standardized'] = df['institution_raw']

print('Country & institution standardization complete')

Country & institution standardization complete


## 5) OECD -> major scientific discipline mapping

In [5]:
mapping_df = pd.read_excel(OECD_MAPPING_XLSX) if os.path.exists(OECD_MAPPING_XLSX) else pd.DataFrame()

if not mapping_df.empty:
    def clean_oecd_category(category):
        if isinstance(category, str):
            parts = category.split()
            if len(parts) > 1: return ' '.join(parts[1:]).lower()
            return category.lower()
        return category
    mapping_df['major_category'] = mapping_df.get('oecddesc', mapping_df.columns[0] if len(mapping_df.columns)>0 else '').apply(clean_oecd_category)
    mapping_df['wos_category'] = mapping_df.get('wosdesc', '').astype(str).str.lower().map(lambda x: collapse_ws(x))
    clean_mapping_df = mapping_df[['major_category', 'wos_category']].dropna().drop_duplicates().reset_index(drop=True)
else:
    clean_mapping_df = pd.DataFrame(columns=['major_category','wos_category'])

# fuzzy matching (optional)
try:
    from fuzzywuzzy import fuzz, process
    FUZZY_AVAILABLE = True
except Exception:
    FUZZY_AVAILABLE = False

def find_best_major_category(scientific_discipline, mapping_df, partial_threshold=60):
    if scientific_discipline is None: return 'unknown'
    d = collapse_ws(str(scientific_discipline).lower())
    if d in {'', '-', 'na', 'n/a', 'null', 'none', 'unknown'}: return 'unknown'
    wos_cats = mapping_df['wos_category'].tolist() if not mapping_df.empty else []
    subs_1 = [w for w in wos_cats if w in d]
    if subs_1:
        best = max(subs_1, key=len)
        return mapping_df[mapping_df['wos_category'] == best]['major_category'].iloc[0]
    subs_2 = [w for w in wos_cats if d in w]
    if subs_2:
        best = min(subs_2, key=len)
        return mapping_df[mapping_df['wos_category'] == best]['major_category'].iloc[0]
    if FUZZY_AVAILABLE and wos_cats:
        best_match = process.extractOne(d, wos_cats, scorer=fuzz.token_sort_ratio)
        if best_match and best_match[1] >= partial_threshold:
            matched = best_match[0]
            return mapping_df[mapping_df['wos_category'] == matched]['major_category'].iloc[0]
    return 'unknown'

if 'Scientific_Discipline' in df.columns:
    df['major_scientific_discipline'] = df['Scientific_Discipline'].apply(lambda x: find_best_major_category(x, clean_mapping_df))
else:
    df['major_scientific_discipline'] = 'unknown'

print('Mapping complete. Unique majors:', df['major_scientific_discipline'].nunique())

Mapping complete. Unique majors: 6


In [6]:
df

ID  Workshop Evaluation  Gender       Status  \
0    9208  Cameroon          R    Male      Pre-Doc   
1    4358     Kenya          R    Male      Pre-Doc   
2    7084     Kenya          S    Male  PhD Student   
3    8849      Mali          R    Male      Pre-Doc   
4    3158     Kenya          R    Male      Pre-Doc   
..    ...       ...        ...     ...          ...   
303  1011      Mali          S    Male      Pre-Doc   
304  5626     Kenya          R  Female      Pre-Doc   
305  1727     Kenya          R    Male      Pre-Doc   
306  8008     Kenya          S  Female  PhD Student   
307  5825     Kenya          S    Male     Post-Doc   

                                 Scientific_Discipline  Institutional_ID  \
0                                    Organic Chemistry               NaN   
1                                             Pharmacy               NaN   
2                                  Medicinal Chemistry               NaN   
3                                              Unknown               NaN   
4    Bioinformatics, Zoology, Cell Biology and Gene...               NaN   
..                                                 ...               ...   
303                                 Structural Biology               NaN   
304                    Bioinformatics and Biochemistry               NaN   
305  Medical Laboratories Sciences and Microbiology...               NaN   
306                                      Biotechnology               NaN   
307       Medical Microbiology and Infectious Diseases               NaN   

                    Primary_ institutional_affiliation  \
0             Prince Abubakar Audu University, Nigeria   
1                                              Unknown   
2                   Nazi Boni University, Burkina Faso   
3    Université des Sciences, des Techniques et des...   
4                      Lagos State University, Nigeria   
..                                                 ...   
303                                            Unknown   
304                            -, United Arab Emriates   
305                 Alzaiem Alazhari University, Sudan   
306                   Ahmadu Bello University, Nigeria   
307                       University of Nairobi, Kenya   

    Country_of_primary_affiliation           country_raw  \
0                          Nigeria               Nigeria   
1                          Unknown               Unknown   
2                     Burkina Faso          Burkina Faso   
3                             Mali                  Mali   
4                          Nigeria               Nigeria   
..                             ...                   ...   
303                        Unknown               Unknown   
304           United Arab Emirates  United Arab Emirates   
305                          Sudan                 Sudan   
306                        Nigeria               Nigeria   
307                          Kenya                 Kenya   

                                       institution_raw      country_name_std  \
0             Prince Abubakar Audu University, Nigeria               Nigeria   
1                                              Unknown               Unknown   
2                   Nazi Boni University, Burkina Faso          Burkina Faso   
3    Université des Sciences, des Techniques et des...                  Mali   
4                      Lagos State University, Nigeria               Nigeria   
..                                                 ...                   ...   
303                                            Unknown               Unknown   
304                            -, United Arab Emriates  United Arab Emirates   
305                 Alzaiem Alazhari University, Sudan                 Sudan   
306                   Ahmadu Bello University, Nigeria               Nigeria   
307                       University of Nairobi, Kenya                 Kenya   

    country_code_std           region  \
0            

## 6) Export cleaned dataset

In [9]:
# Pre-export transformations (separate cell):
# 1) Replace primary columns with standardized values where available
if 'Primary_ institutional_affiliation' in df.columns and 'institution_name_standardized' in df.columns:
    df['Primary_ institutional_affiliation'] = df['institution_name_standardized']
if 'Country_of_primary_affiliation' in df.columns and 'country_name_std' in df.columns:
    df['Country_of_primary_affiliation'] = df['country_name_std']

# 2) Drop intermediate/unwanted columns (be defensive about possible spellings)
cols_to_drop = [
    'Institutional_ID', 'country_raw', 'institution_raw', 'country_name_std', 'country_code_std',
    'Instuition_name_standardized', 'institution_name_standardized', 'Institution_name_standardized'
]
cols_present = [c for c in cols_to_drop if c in df.columns]
if cols_present:
    df.drop(columns=cols_present, inplace=True)

# 3) Capitalize major_scientific_discipline
if 'major_scientific_discipline' in df.columns:
    df['major_scientific_discipline'] = df['major_scientific_discipline'].astype(str).str.capitalize()

# 4) Replace empty strings with NaN then fill all NaNs with 'Unknown'
df.replace('', np.nan, inplace=True)
df.fillna('Unknown', inplace=True)

print('Pre-export transformations applied. Columns now:', list(df.columns))

Pre-export transformations applied. Columns now: ['ID', 'Workshop', 'Evaluation', 'Gender', 'Status', 'Scientific_Discipline', 'Primary_ institutional_affiliation', 'Country_of_primary_affiliation', 'region', 'major_scientific_discipline']


In [11]:
# Export final cleaned dataset to CSV
df.to_csv(OUT_PATH, index=False)
print('Exported cleaned dataset to', OUT_PATH)
print('Final shape:', df.shape)

Exported cleaned dataset to processed_data/cleansed_biostruct_data_v3.csv
Final shape: (308, 10)


### Notes
- This v3 notebook is intended to be the clean, single-pass script.
- If you want me to run the notebook cells here to validate they execute end-to-end, I can do that next.
- For production, extract core functions into a module and add unit tests.